大数据挖掘最深入人心的一个故事应该是“啤酒与尿布”，这个规律就是用关联分析发现的。今天我们就来点关联分析，刚刚学到的，现学现分享😄。下面假设是某超市的交易记录（我编造的）

交易id|购买商品
---|---
0|牛奶，洋葱，猪肉，鸡蛋，洋葱，酸奶
1|洋葱，豆角，酸奶， 鸡蛋， 苹果
2|牛奶，苹果，豆角，鸡蛋
3|牛奶，玉米，胡萝卜，豆角，酸奶
4|玉米，洋葱， 豆角， 冰激凌， 鸡蛋


### 基本概念
我们在关联分析之前先了解几个基本概念。

找出频繁一起出现的物品集的集合，我们称之为频繁项集。比如一个超市的频繁项集可能有{{啤酒,尿布},{鸡蛋,牛奶},{香蕉,苹果}}

概念|计算公式|作用
---|---|---
**支持度(Support)**| ``支持度 = (包含物品A的记录数量)/(总的记录数量)``  |商品的流行程度
**置信度(Confidence)**|``置信度(A->B) = (同时包含物品A和B的记录数量)/(包含 A 的记录数量)``|如果购买A，有多大的可能性买B  
**提升度(Lift)**| ``提升度(A->B) = 置信度(A->B)/(支持度 A)``|当销售A时，另一个产品B销售会增加多少


### mlxtend安装
mlxtend是python的机器学习扩展库，在数据科学中也会经常遇到。在本文主要是使用其中的关联分析一些方法

In [3]:
!pip3 install mlxtend

### 编码
这块跟sklearn文本分析的contvertorizer和tfidfvectorizer差不多，都是从数据中学到 空间

In [15]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

#测试数据
records = [['牛奶', '洋葱', '猪肉', '鸡蛋', '洋葱', '酸奶'],
           ['洋葱', '豆角', '酸奶', '鸡蛋', '苹果'],
           ['牛奶', '苹果', '豆角', '鸡蛋'],
           ['牛奶', '玉米', '胡萝卜', '豆角', '酸奶'],
           ['玉米', '洋葱', '豆角', '冰激凌', '鸡蛋']]

Encoder = TransactionEncoder()
encoded_data = Encoder.fit_transform(records)
df = pd.DataFrame(encoded_data, columns=Encoder.columns_)
df

,冰激凌,洋葱,牛奶,猪肉,玉米,胡萝卜,苹果,豆角,酸奶,鸡蛋
0,False,True,True,True,False,False,False,False,True,True
1,False,True,False,False,False,False,True,True,True,True
2,False,False,True,False,False,False,True,True,False,True
3,False,False,True,False,True,True,False,True,True,False
4,True,True,False,False,True,False,False,True,False,True


### 频繁项
找出频繁项,这里会用到 ``mlxtend.frequent_patterns`` 中的  ``apriori`` 函数

apriori(df, min_support=0.5,  use_colnames=False, max_len=None)
- df：编码后的dataframe，如上图
- min_support：给定的最小支持度
- use_colnames：默认False，则返回的物品组会用编号显示，为True的话直接显示物品名称。
- max_len=None：最大物品组合数，默认是None，不做限制。

我们设置min_support=0.05， use_colnames=True， 需要计算最多四个物品组合的话，便将max_len这个值设置为4。

In [26]:
from mlxtend.frequent_patterns import apriori

frequent_items = apriori(df, min_support=0.05, use_colnames=True, max_len=4).sort_values(by='support', ascending=False)
frequent_items

,support,itemsets
7,0.8,(豆角)
9,0.8,(鸡蛋)
1,0.6,(洋葱)
2,0.6,(牛奶)
40,0.6,"(鸡蛋, 豆角)"
...,...,...
44,0.2,"(鸡蛋, 洋葱, 冰激凌)"
43,0.2,"(豆角, 洋葱, 冰激凌)"
42,0.2,"(玉米, 洋葱, 冰激凌)"
37,0.2,"(苹果, 酸奶)"


### 求关联规则
``mlxtend.frequent_patterns``里封装有关联规则函数 ``association_rules`` ，我们可以直接使用该函数帮助我们找到关联规则

association_rules(df, metric='confidence', min_threshold=0.8)
- df: 频繁项dataframe
- metric：默认是confidence
- min_threshold：给选定的metric设定最低阈值

In [29]:
from mlxtend.frequent_patterns import association_rules

ass_rule = association_rules(frequent_items, metric='confidence', min_threshold=0.8)

ass_rule.sort_values(by='leverage', ascending=False, inplace=True)  

ass_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(苹果),"(鸡蛋, 豆角)",0.4,0.6,0.4,1.0,1.666667,0.16,inf
7,"(鸡蛋, 酸奶)",(洋葱),0.4,0.6,0.4,1.0,1.666667,0.16,inf
71,"(牛奶, 胡萝卜)","(玉米, 酸奶)",0.2,0.2,0.2,1.0,5.000000,0.16,inf
75,"(玉米, 牛奶, 豆角)",(胡萝卜),0.2,0.2,0.2,1.0,5.000000,0.16,inf
175,"(玉米, 洋葱)","(冰激凌, 豆角)",0.2,0.2,0.2,1.0,5.000000,0.16,inf
...,...,...,...,...,...,...,...,...,...
47,"(牛奶, 苹果, 豆角)",(鸡蛋),0.2,0.8,0.2,1.0,1.250000,0.04,inf
202,"(洋葱, 苹果)",(鸡蛋),0.2,0.8,0.2,1.0,1.250000,0.04,inf
38,"(玉米, 胡萝卜, 酸奶)",(豆角),0.2,0.8,0.2,1.0,1.250000,0.04,inf
205,"(洋葱, 苹果)",(豆角),0.2,0.8,0.2,1.0,1.250000,0.04,inf


从上面看，买了苹果的人，会再买鸡蛋和豆角。可能这人爱吃苹果，主食会来来豆角炒鸡蛋😄

编的数据，所以解读就当笑话吧～